# How would ReproNim do that?
## Volume 1: nifti file: container: result: publish

This is the companion Jupyter notebook to [*How would ReproNim do that Volume 1*](https://docs.google.com/document/d/1L4ODHVp36NilWQRi3gDYQK_N58ZXpMKqHagv0ZmtkbA/edit#heading=h.1z04rarws9nl).

*Note: this is a limited notebook.  Its notes are sparser than those found in the source document, no images are generated, and it stops short of the data publishing step.*

Common runtimes are under one minute for `datalad containers-add`, 15 minutes for the first kwyk run, and 25 minutes for the second kwyk run.

### Step 1: Your image file should be under content management.

We check the datalad version, then create the datalad project.  We then copy an image file to the project directory and put it under content management using datalad.

In [ ]:
datalad --version

In [ ]:
datalad create reprokwyk

In [ ]:
cd reprokwyk

In [ ]:
cp ../anat.nii .

In [ ]:
datalad add -m "Added my favorite anatomy file" anat.nii

### Step 2: Your container should be under content management.

We put the kwyk container that we intend to use under content management using datalad.

In [ ]:
datalad containers-add -i kwyk-img -u dhub://neuronets/kwyk:latest-cpu kwyk

We check the version of the kwyk container, first using docker directly, then using datalad.

In [ ]:
docker run -it --rm neuronets/kwyk:latest-cpu --version

In [ ]:
datalad containers-run -n kwyk -- --version

### Step 3: Have datalad run the container on the data.

In [ ]:
datalad containers-run -n kwyk --input anat.nii --output kwyk-output.nii.gz '{inputs}' '{outputs}'

In this example, we generated two output files:
* kwyk-output.nii.gz_means.ini.gz
* kwyk-output.nii.gz_means_orig.nii.gz
These are then automatically put under content management by datalad (`add (ok: 2)`).

We try again using the bvwm_multi_prior model for prediction, but first we must remove the output files.  Since this is a content management task, we do that using datalad.

In [ ]:
datalad remove kwyk-output* --nocheck

Now we rerun the analysis using the bvwm_multi_prior model.

In [ ]:
datalad containers-run -n kwyk --input anat.nii --output "kwyk-output*" -- -m bvwn_multi_prior '{inputs}' 'kwyk-output.nii.gz'

Let's see what happened and what datalad gave us over running this in the traditional manner.

We use git to see the history of an output, `kwyk-output.nii.gz_means_orig.nii.gz`, and see that it was created using our first run, then removed, then created again during our second run.

The history is shown with the latest event first.

In [ ]:
git log kwyk-output.nii.gz_means_orig.nii